In [1]:
import numpy as np
import cv2
from PIL import Image
import open3d as o3d
import json
import os
from tqdm import tqdm

from sklearn.cluster import DBSCAN

import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from matplotlib import rcParams
from shapely.geometry import MultiPoint, box

# from pyquaternion.quaternion import Quaternion
from pyquaternion import Quaternion
import os.path as osp
from nuscenes import NuScenes

# Utils for Lidar and Radar
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.geometry_utils import view_points, BoxVisibility
from nuscenes.utils.data_classes import LidarPointCloud
from nuscenes.utils.data_classes import RadarPointCloud
from nuscenes.scripts.export_2d_annotations_as_json import get_2d_boxes, post_process_coords
from utils import *

from typing import Tuple, List, Dict, Union

%matplotlib inline

In [2]:
#nusc = NuScenes(version='v1.0-mini', dataroot='/home/gus/Documents/AI_S/nuScenes/v1_0-mini', verbose=True)
nusc = NuScenes(version='v1.0-mini', dataroot='/home/sherlock/Documents/nuScenes/data', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.4 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [3]:
def parse_vfh_to_numpy(path):
    # Reading the file
    file = open(path,"r")
    file = file.read()
    # Replace undesired elements
    file = file.replace('(', '').replace(')', '').replace(' ', '')
    # Split the string and create a numpy array with floats
    descriptor_array = np.array(file.split(',')).astype(float)
    
    return descriptor_array

In [6]:
arr = parse_vfh_to_numpy("dataset/point_features/pcd_segmentation_0a1e2d4329c046cbb1e6046515936b34.txt")
arr.shape

(640,)

In [7]:
arr

array([6.39326e-04, 4.50408e-04, 5.90212e-04, 7.26769e-04, 5.80368e-04,
       4.73970e-04, 4.71352e-04, 3.59405e-04, 3.55321e-04, 3.41707e-04,
       2.96781e-04, 2.75313e-04, 2.86519e-04, 2.10595e-04, 2.23057e-04,
       2.01903e-04, 2.02427e-04, 2.24000e-04, 1.73838e-04, 1.55931e-04,
       1.86614e-04, 1.49752e-04, 1.81587e-04, 1.70801e-04, 1.35824e-04,
       1.49543e-04, 1.29331e-04, 1.25457e-04, 1.44935e-04, 1.09225e-04,
       1.12157e-04, 1.13414e-04, 9.60298e-05, 1.00428e-04, 8.28349e-05,
       1.22943e-04, 9.36212e-05, 8.61860e-05, 8.25207e-05, 7.57138e-05,
       6.61841e-05, 8.73379e-05, 8.00074e-05, 7.78082e-05, 7.47713e-05,
       7.17344e-05, 7.13155e-05, 7.77035e-05, 5.92725e-05, 8.27302e-05,
       8.62907e-05, 6.91163e-05, 6.69172e-05, 7.84366e-05, 7.99027e-05,
       8.27302e-05, 5.77017e-05, 9.86479e-05, 8.02168e-05, 1.04722e-04,
       8.60813e-05, 9.03748e-05, 8.01121e-05, 5.24656e-05, 3.25685e-05,
       5.78064e-05, 8.47199e-05, 8.55576e-05, 8.02168e-05, 9.079